# Task description
- Classify the speakers of given features.
- Main goal: Learn how to use transformer.
- Baselines:
  - Easy: Run sample code and know how to use transformer.
  - Medium: Know how to adjust parameters of transformer.
  - Strong: Construct [conformer](https://arxiv.org/abs/2005.08100) which is a variety of transformer. 
  - Boss: Implement [Self-Attention Pooling](https://arxiv.org/pdf/2008.01077v1.pdf) & [Additive Margin Softmax](https://arxiv.org/pdf/1801.05599.pdf) to further boost the performance.

- Other links
  - Competiton: [link](https://www.kaggle.com/t/49ea0c385a974db5919ec67299ba2e6b)
  - Slide: [link](https://docs.google.com/presentation/d/1LDAW0GGrC9B6D7dlNdYzQL6D60-iKgFr/edit?usp=sharing&ouid=104280564485377739218&rtpof=true&sd=true)
  - Data: [link](https://github.com/googly-mingto/ML2023HW4/releases)

# Download dataset
- Data is [here](https://drive.google.com/drive/folders/1vI1kuLB-q1VilIftiwnPOCAeOOFfBZge?usp=sharing)

In [ ]:
!wget https://github.com/googly-mingto/ML2023HW4/releases/download/data/Dataset.tar.gz.partaa
!wget https://github.com/googly-mingto/ML2023HW4/releases/download/data/Dataset.tar.gz.partab
!wget https://github.com/googly-mingto/ML2023HW4/releases/download/data/Dataset.tar.gz.partac
!wget https://github.com/googly-mingto/ML2023HW4/releases/download/data/Dataset.tar.gz.partad

!cat Dataset.tar.gz.part* > Dataset.tar.gz
!rm Dataset.tar.gz.partaa
!rm Dataset.tar.gz.partab
!rm Dataset.tar.gz.partac
!rm Dataset.tar.gz.partad
# unzip the file
!tar zxf Dataset.tar.gz
!rm Dataset.tar.gz

In [ ]:
!tar zxf Dataset.tar.gz

In [20]:
import numpy as np
import torch
import random

def set_seed(seed):
    np.random.seed(seed)
    random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True

set_seed(87)

# Data

## Dataset
- Original dataset is [Voxceleb2](https://www.robots.ox.ac.uk/~vgg/data/voxceleb/vox2.html).
- The [license](https://creativecommons.org/licenses/by/4.0/) and [complete version](https://www.robots.ox.ac.uk/~vgg/data/voxceleb/files/license.txt) of Voxceleb2.
- We randomly select 600 speakers from Voxceleb2.
- Then preprocess the raw waveforms into mel-spectrograms.

- Args:
  - data_dir: The path to the data directory.
  - metadata_path: The path to the metadata.
  - segment_len: The length of audio segment for training. 
- The architecture of data directory \\
  - data directory \\
  |---- metadata.json \\
  |---- testdata.json \\
  |---- mapping.json \\
  |---- uttr-{random string}.pt \\

- The information in metadata
  - "n_mels": The dimention of mel-spectrogram.
  - "speakers": A dictionary. 
    - Key: speaker ids.
    - value: "feature_path" and "mel_len"


For efficiency, we segment the mel-spectrograms into segments in the traing step.

In [21]:
'''
import os
import json
import torch
import random
from pathlib import Path
from torch.utils.data import Dataset
from torch.nn.utils.rnn import pad_sequence
 
 
class myDataset(Dataset):
	def __init__(self, data_dir, segment_len=128):
		self.data_dir = data_dir
		self.segment_len = segment_len
	
		# Load the mapping from speaker neme to their corresponding id. 
		mapping_path = Path(data_dir) / "mapping.json"
		mapping = json.load(mapping_path.open())
		self.speaker2id = mapping["speaker2id"]
	
		# Load metadata of training data.
		metadata_path = Path(data_dir) / "metadata.json"
		metadata = json.load(open(metadata_path))["speakers"]
	
		# Get the total number of speaker.
		self.speaker_num = len(metadata.keys())
		self.data = []
		for speaker in metadata.keys():
			for utterances in metadata[speaker]:
				self.data.append([utterances["feature_path"], self.speaker2id[speaker]])
 
	def __len__(self):
			return len(self.data)
 
	def __getitem__(self, index):
		feat_path, speaker = self.data[index]
		# Load preprocessed mel-spectrogram.
		mel = torch.load(os.path.join(self.data_dir, feat_path))

		# Segmemt mel-spectrogram into "segment_len" frames.
		if len(mel) > self.segment_len:
			# Randomly get the starting point of the segment.
			start = random.randint(0, len(mel) - self.segment_len)
			# Get a segment with "segment_len" frames.
			mel = torch.FloatTensor(mel[start:start+self.segment_len])
		else:
			mel = torch.FloatTensor(mel)
		# Turn the speaker id into long for computing loss later.
		speaker = torch.FloatTensor([speaker]).long()
		return mel, speaker
 
	def get_speaker_number(self):
		return self.speaker_num
'''
from mydataset import myDataset

## Dataloader
- Split dataset into training dataset(90%) and validation dataset(10%).
- Create dataloader to iterate the data.

In [22]:
import torch
from torch.utils.data import DataLoader, random_split
from torch.nn.utils.rnn import pad_sequence


'''
def collate_batch(batch):
	# Process features within a batch.
	"""Collate a batch of data."""
	mel, speaker = zip(*batch)
	# Because we train the model batch by batch, we need to pad the features in the same batch to make their lengths the same.
	mel = pad_sequence(mel, batch_first=True, padding_value=-20)    # pad log 10^(-20) which is very small value.
	# mel: (batch size, length, 40)
	return mel, torch.FloatTensor(speaker).long()
'''
from collate_batch import collate_batch


def get_dataloader(data_dir, batch_size, n_workers):
	"""Generate dataloader"""
	dataset = myDataset(data_dir)
	print("Colin ctor myDataset")
	speaker_num = dataset.get_speaker_number()
	# Split dataset into training dataset and validation dataset
	trainlen = int(0.9 * len(dataset))
	lengths = [trainlen, len(dataset) - trainlen]
	trainset, validset = random_split(dataset, lengths)
	print("Colin finish split")
	train_loader = DataLoader(
		trainset,
		batch_size=batch_size,
		shuffle=True,
		drop_last=True,
		num_workers=n_workers,
		pin_memory=True,
		collate_fn=collate_batch,
	)
	print("Colin finish ctor train_loader")
	valid_loader = DataLoader(
		validset,
		batch_size=batch_size,
		num_workers=n_workers,
		drop_last=True,
		pin_memory=True,
		collate_fn=collate_batch,
	)
	print("Colin finish ctor valid_loader")
	return train_loader, valid_loader, speaker_num

# Model
- TransformerEncoderLayer:
  - Base transformer encoder layer in [Attention Is All You Need](https://arxiv.org/abs/1706.03762)
  - Parameters:
    - d_model: the number of expected features of the input (required).

    - nhead: the number of heads of the multiheadattention models (required).

    - dim_feedforward: the dimension of the feedforward network model (default=2048).

    - dropout: the dropout value (default=0.1).

    - activation: the activation function of intermediate layer, relu or gelu (default=relu).

- TransformerEncoder:
  - TransformerEncoder is a stack of N transformer encoder layers
  - Parameters:
    - encoder_layer: an instance of the TransformerEncoderLayer() class (required).

    - num_layers: the number of sub-encoder-layers in the encoder (required).

    - norm: the layer normalization component (optional).

In [23]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchaudio


class Classifier(nn.Module):
	def __init__(self, d_model=320, n_spks=600, dropout=0.1):
		super().__init__()
		# Project the dimension of features from that of input into d_model.
		self.prenet = nn.Linear(40, d_model)
		# TODO:
		#   Change Transformer to Conformer.
		#   https://arxiv.org/abs/2005.08100
		#self.encoder_layer = nn.TransformerEncoderLayer(
		#	d_model=d_model, dim_feedforward=256, nhead=32
		#)
		# self.encoder = nn.TransformerEncoder(self.encoder_layer, num_layers=2)
		self.encoder = torchaudio.models.Conformer(
			input_dim=d_model, num_heads=4, ffn_dim=256, num_layers=1, depthwise_conv_kernel_size=15
		)

		# Project the the dimension of features from d_model into speaker nums.
		self.pred_layer = nn.Sequential(
			nn.Linear(d_model, d_model),
			nn.ReLU(),
			nn.Linear(d_model, n_spks),
		)

	def forward(self, mels):
		"""
		args:
			mels: (batch size, length, 40)
		return:
			out: (batch size, n_spks)
		"""
		out = self.prenet(mels)  # out: (batch size, length, d_model)
		"""
		out = out.permute(1, 0, 2) # out: (length, batch size, d_model)
		# The encoder layer expect features in the shape of (length, batch size, d_model).
		out = self.encoder_layer(out)
		# out = self.encoder(out)
		# out: (batch size, length, d_model)
		out = out.transpose(0, 1)
		"""
		lengths = torch.full((mels.size(0),), mels.size(1), dtype=torch.long).to(out.device)
		out, _ = self.encoder(out, lengths) #out: (batch size, length, d_model)

		# mean pooling
		stats = out.mean(dim=1)

		# out: (batch, n_spks)
		out = self.pred_layer(stats)
		return out

# Learning rate schedule
- For transformer architecture, the design of learning rate schedule is different from that of CNN.
- Previous works show that the warmup of learning rate is useful for training models with transformer architectures.
- The warmup schedule
  - Set learning rate to 0 in the beginning.
  - The learning rate increases linearly from 0 to initial learning rate during warmup period.

In [24]:
import math

import torch
from torch.optim import Optimizer
from torch.optim.lr_scheduler import LambdaLR


def get_cosine_schedule_with_warmup(
	optimizer: Optimizer,
	num_warmup_steps: int,
	num_training_steps: int,
	num_cycles: float = 0.5,
	last_epoch: int = -1,
):
	"""
	Create a schedule with a learning rate that decreases following the values of the cosine function between the
	initial lr set in the optimizer to 0, after a warmup period during which it increases linearly between 0 and the
	initial lr set in the optimizer.

	Args:
		optimizer (:class:`~torch.optim.Optimizer`):
		The optimizer for which to schedule the learning rate.
		num_warmup_steps (:obj:`int`):
		The number of steps for the warmup phase.
		num_training_steps (:obj:`int`):
		The total number of training steps.
		num_cycles (:obj:`float`, `optional`, defaults to 0.5):
		The number of waves in the cosine schedule (the defaults is to just decrease from the max value to 0
		following a half-cosine).
		last_epoch (:obj:`int`, `optional`, defaults to -1):
		The index of the last epoch when resuming training.

	Return:
		:obj:`torch.optim.lr_scheduler.LambdaLR` with the appropriate schedule.
	"""
	def lr_lambda(current_step):
		# Warmup
		if current_step < num_warmup_steps:
			return float(current_step) / float(max(1, num_warmup_steps))
		# decadence
		progress = float(current_step - num_warmup_steps) / float(
			max(1, num_training_steps - num_warmup_steps)
		)
		return max(
			0.0, 0.5 * (1.0 + math.cos(math.pi * float(num_cycles) * 2.0 * progress))
		)

	return LambdaLR(optimizer, lr_lambda, last_epoch)

# Model Function
- Model forward function.

In [25]:
import torch


def model_fn(batch, model, criterion, device):
	"""Forward a batch through the model."""

	mels, labels = batch
	mels = mels.to(device)
	labels = labels.to(device)

	outs = model(mels)

	loss = criterion(outs, labels)

	# Get the speaker id with highest probability.
	preds = outs.argmax(1)
	# Compute accuracy.
	accuracy = torch.mean((preds == labels).float())

	return loss, accuracy

# Validate
- Calculate accuracy of the validation set.

In [26]:
from tqdm import tqdm
import torch


def valid(dataloader, model, criterion, device): 
	"""Validate on validation set."""

	model.eval()
	running_loss = 0.0
	running_accuracy = 0.0
	pbar = tqdm(total=len(dataloader.dataset), ncols=0, desc="Valid", unit=" uttr")

	for i, batch in enumerate(dataloader):
		with torch.no_grad():
			loss, accuracy = model_fn(batch, model, criterion, device)
			running_loss += loss.item()
			running_accuracy += accuracy.item()

		pbar.update(dataloader.batch_size)
		pbar.set_postfix(
			loss=f"{running_loss / (i+1):.2f}",
			accuracy=f"{running_accuracy / (i+1):.2f}",
		)

	pbar.close()
	model.train()

	return running_accuracy / len(dataloader)

# Main function

In [27]:
from tqdm import tqdm

import torch
import torch.nn as nn
from torch.optim import AdamW
from torch.utils.data import DataLoader, random_split


def parse_args():
	"""arguments"""
	config = {
		"data_dir": ".\\Dataset",
		"save_path": "model.ckpt",
		"batch_size": 32,
		"n_workers": 8,
		"valid_steps": 2000,
		"warmup_steps": 1000,
		"save_steps": 10000,
		"total_steps": 1000000,
	}

	return config


def main(
	data_dir,
	save_path,
	batch_size,
	n_workers,
	valid_steps,
	warmup_steps,
	total_steps,
	save_steps,
):
	"""Main function."""
	device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
	print(f"[Info]: Use {device} now!")

	train_loader, valid_loader, speaker_num = get_dataloader(data_dir, batch_size, n_workers)
	print("Colin after get_dataloader")
	train_iterator = iter(train_loader)
	print(f"[Info]: Finish loading data!",flush = True)

	model = Classifier(n_spks=speaker_num).to(device)
	criterion = nn.CrossEntropyLoss()
	optimizer = AdamW(model.parameters(), lr=1e-3)
	scheduler = get_cosine_schedule_with_warmup(optimizer, warmup_steps, total_steps)
	print(f"[Info]: Finish creating model!",flush = True)

	best_accuracy = -1.0
	best_state_dict = None

	pbar = tqdm(total=valid_steps, ncols=0, desc="Train", unit=" step")

	for step in range(total_steps):
		# Get data
		try:
			batch = next(train_iterator)
		except StopIteration:
			train_iterator = iter(train_loader)
			batch = next(train_iterator)

		loss, accuracy = model_fn(batch, model, criterion, device)
		batch_loss = loss.item()
		batch_accuracy = accuracy.item()

		# Updata model
		loss.backward()
		optimizer.step()
		scheduler.step()
		optimizer.zero_grad()

		# Log
		pbar.update()
		pbar.set_postfix(
			loss=f"{batch_loss:.2f}",
			accuracy=f"{batch_accuracy:.2f}",
			step=step + 1,
		)

		# Do validation
		if (step + 1) % valid_steps == 0:
			pbar.close()

			valid_accuracy = valid(valid_loader, model, criterion, device)

			# keep the best model
			if valid_accuracy > best_accuracy:
				best_accuracy = valid_accuracy
				best_state_dict = model.state_dict()

			pbar = tqdm(total=valid_steps, ncols=0, desc="Train", unit=" step")

		# Save the best model so far.
		if (step + 1) % save_steps == 0 and best_state_dict is not None:
			torch.save(best_state_dict, save_path)
			pbar.write(f"Step {step + 1}, best model saved. (accuracy={best_accuracy:.4f})")

	pbar.close()


if __name__ == "__main__":
	main(**parse_args())

[Info]: Use cuda now!
Colin ctor myDataset
Colin finish split
Colin finish ctor train_loader
Colin finish ctor valid_loader
Colin after get_dataloader
[Info]: Finish loading data!
[Info]: Finish creating model!


Train: 100% 2000/2000 [00:57<00:00, 34.55 step/s, accuracy=0.34, loss=3.17, step=2000]
Valid: 100% 5664/5667 [00:24<00:00, 232.89 uttr/s, accuracy=0.26, loss=3.48] 
Train: 100% 2000/2000 [00:57<00:00, 35.03 step/s, accuracy=0.41, loss=2.83, step=4000]
Valid: 100% 5664/5667 [00:25<00:00, 226.49 uttr/s, accuracy=0.37, loss=2.78] 
Train: 100% 2000/2000 [00:56<00:00, 35.25 step/s, accuracy=0.38, loss=2.49, step=6000]
Valid: 100% 5664/5667 [00:23<00:00, 242.03 uttr/s, accuracy=0.46, loss=2.37] 
Train: 100% 2000/2000 [01:28<00:00, 22.69 step/s, accuracy=0.59, loss=1.32, step=8000]
Valid: 100% 5664/5667 [00:22<00:00, 248.51 uttr/s, accuracy=0.53, loss=2.07] 
Train: 100% 2000/2000 [00:57<00:00, 34.65 step/s, accuracy=0.62, loss=1.53, step=1e+4]
Valid: 100% 5664/5667 [00:23<00:00, 241.15 uttr/s, accuracy=0.57, loss=1.91] 
Train:   0% 10/2000 [00:00<00:46, 43.12 step/s, accuracy=0.59, loss=1.50, step=1e+4]

Step 10000, best model saved. (accuracy=0.5674)


Train: 100% 2000/2000 [00:56<00:00, 35.56 step/s, accuracy=0.69, loss=1.63, step=12000]
Valid: 100% 5664/5667 [00:23<00:00, 240.14 uttr/s, accuracy=0.60, loss=1.73] 
Train: 100% 2000/2000 [00:56<00:00, 35.55 step/s, accuracy=0.66, loss=1.14, step=14000]
Valid: 100% 5664/5667 [00:22<00:00, 246.84 uttr/s, accuracy=0.62, loss=1.69] 
Train: 100% 2000/2000 [01:17<00:00, 25.84 step/s, accuracy=0.69, loss=1.70, step=16000]
Valid: 100% 5664/5667 [00:23<00:00, 241.74 uttr/s, accuracy=0.64, loss=1.60] 
Train: 100% 2000/2000 [00:55<00:00, 35.81 step/s, accuracy=0.75, loss=0.80, step=18000]
Valid: 100% 5664/5667 [00:22<00:00, 246.57 uttr/s, accuracy=0.64, loss=1.55] 
Train: 100% 2000/2000 [00:56<00:00, 35.50 step/s, accuracy=0.69, loss=1.11, step=2e+4] 
Valid: 100% 5664/5667 [00:22<00:00, 249.39 uttr/s, accuracy=0.65, loss=1.53] 
Train:   0% 10/2000 [00:00<00:43, 45.70 step/s, accuracy=0.69, loss=1.08, step=2e+4]

Step 20000, best model saved. (accuracy=0.6501)


...

Step 980000, best model saved. (accuracy=0.9126)


Train: 100% 2000/2000 [00:55<00:00, 36.08 step/s, accuracy=1.00, loss=0.00, step=982000]
Valid: 100% 5664/5667 [00:23<00:00, 245.78 uttr/s, accuracy=0.91, loss=0.65] 
Train: 100% 2000/2000 [00:54<00:00, 36.77 step/s, accuracy=1.00, loss=0.00, step=984000]
Valid: 100% 5664/5667 [00:24<00:00, 235.60 uttr/s, accuracy=0.91, loss=0.70] 
Train: 100% 2000/2000 [00:54<00:00, 36.47 step/s, accuracy=1.00, loss=0.00, step=986000]
Valid: 100% 5664/5667 [00:22<00:00, 247.59 uttr/s, accuracy=0.91, loss=0.66] 
Train: 100% 2000/2000 [01:17<00:00, 25.93 step/s, accuracy=0.97, loss=0.03, step=988000]
Valid: 100% 5664/5667 [00:22<00:00, 246.77 uttr/s, accuracy=0.90, loss=0.69] 
Train: 100% 2000/2000 [00:57<00:00, 34.57 step/s, accuracy=1.00, loss=0.00, step=990000]
Valid: 100% 5664/5667 [00:26<00:00, 212.73 uttr/s, accuracy=0.91, loss=0.68] 
Train:   0% 9/2000 [00:00<00:50, 39.38 step/s, accuracy=1.00, loss=0.00, step=990009]

Step 990000, best model saved. (accuracy=0.9126)


Train: 100% 2000/2000 [00:54<00:00, 36.46 step/s, accuracy=1.00, loss=0.00, step=992000]
Valid: 100% 5664/5667 [00:23<00:00, 242.74 uttr/s, accuracy=0.91, loss=0.65] 
Train: 100% 2000/2000 [00:56<00:00, 35.34 step/s, accuracy=1.00, loss=0.00, step=994000]
Valid: 100% 5664/5667 [00:23<00:00, 242.14 uttr/s, accuracy=0.91, loss=0.65] 
Train: 100% 2000/2000 [01:19<00:00, 25.02 step/s, accuracy=1.00, loss=0.00, step=996000]
Valid: 100% 5664/5667 [00:25<00:00, 220.42 uttr/s, accuracy=0.91, loss=0.66] 
Train: 100% 2000/2000 [00:55<00:00, 35.90 step/s, accuracy=1.00, loss=0.00, step=998000]
Valid: 100% 5664/5667 [00:23<00:00, 243.44 uttr/s, accuracy=0.90, loss=0.68] 
Train: 100% 2000/2000 [00:58<00:00, 34.05 step/s, accuracy=1.00, loss=0.00, step=1e+6]  
Valid: 100% 5664/5667 [00:25<00:00, 219.08 uttr/s, accuracy=0.90, loss=0.68] 
Train:   0% 0/2000 [00:00<?, ? step/s]


Step 1000000, best model saved. (accuracy=0.9126)


# Inference

## Dataset of inference

In [28]:
import os
import json
import torch
from pathlib import Path
from torch.utils.data import Dataset


'''
class InferenceDataset(Dataset):
	def __init__(self, data_dir):
		testdata_path = Path(data_dir) / "testdata.json"
		metadata = json.load(testdata_path.open())
		self.data_dir = data_dir
		self.data = metadata["utterances"]

	def __len__(self):
		return len(self.data)

	def __getitem__(self, index):
		utterance = self.data[index]
		feat_path = utterance["feature_path"]
		mel = torch.load(os.path.join(self.data_dir, feat_path))

		return feat_path, mel

def inference_collate_batch(batch):
	"""Collate a batch of data."""
	feat_paths, mels = zip(*batch)

	return feat_paths, torch.stack(mels)

'''
from inference_utility import InferenceDataset, inference_collate_batch


## Main funcrion of Inference

In [29]:
import json
import csv
from pathlib import Path
from tqdm.notebook import tqdm

import torch
from torch.utils.data import DataLoader

def parse_args():
	"""arguments"""
	config = {
		"data_dir": ".\\Dataset",
		"model_path": ".\\model.ckpt",
		"output_path": ".\\output.csv",
	}

	return config


def main(
	data_dir,
	model_path,
	output_path,
):
	"""Main function."""
	device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
	print(f"[Info]: Use {device} now!")

	mapping_path = Path(data_dir) / "mapping.json"
	mapping = json.load(mapping_path.open())

	dataset = InferenceDataset(data_dir)
	dataloader = DataLoader(
		dataset,
		batch_size=1,
		shuffle=False,
		drop_last=False,
		num_workers=8,
		collate_fn=inference_collate_batch,
	)
	print(f"[Info]: Finish loading data!",flush = True)

	speaker_num = len(mapping["id2speaker"])
	model = Classifier(n_spks=speaker_num).to(device)
	model.load_state_dict(torch.load(model_path))
	model.eval()
	print(f"[Info]: Finish creating model!",flush = True)

	results = [["Id", "Category"]]
	for feat_paths, mels in tqdm(dataloader):
		with torch.no_grad():
			mels = mels.to(device)
			outs = model(mels)
			preds = outs.argmax(1).cpu().numpy()
			for feat_path, pred in zip(feat_paths, preds):
				results.append([feat_path, mapping["id2speaker"][str(pred)]])

	with open(output_path, 'w', newline='') as csvfile:
		writer = csv.writer(csvfile)
		writer.writerows(results)


if __name__ == "__main__":
	main(**parse_args())

[Info]: Use cuda now!
[Info]: Finish loading data!
[Info]: Finish creating model!


  0%|          | 0/8000 [00:00<?, ?it/s]